Another run through using Director Film Dataframe to see if there are new movies to add and to add IMDB id to director film list

In [287]:
import pandas as pd
from imdb import Cinemagoer
import re

df_director_films = pd.read_csv('director_films_clean.csv', encoding= 'unicode_escape')
df_director_names = pd.read_csv('director_names.csv', encoding='unicode_escape')
dfClean = pd.DataFrame(columns=['Movie_ID', 'Title', 'Year', 'Person_ID', 'Name'])

In [3]:
ia = Cinemagoer()
found = False

for i in range(0, len(df_director_names.index)):
    people = ia.search_person(df_director_names['Name'][i])
    found = False
    for person in people:
        if found == False:
            #print(person['name'])        
            works = ia.get_person(person.personID)
            #print (works['filmography'].keys())
            if works.has_key('director'):
                for job in works['filmography'].keys():
                    if job == 'director': 
                        #print('Person ID:', person.personID, '\tDirector:', person['name'], '\t# Job:', job, )
                        for movie in works['filmography'][job]:
                            #if 'year' in movie:
                                #print (movie['title'], movie['year'], '\n')
                            for row in df_director_films.itertuples(index=False):
                                if row.Movie == movie['title'] and row.Year == movie['year'] and found == False:
                                    found = True
                                    for m in works['filmography'][job]:
                                        year = 0000 if 'year' not in m else int(m['year']) 
                                        #print(m['title'],'\t', m['year'])
                                        dfClean.loc[len(dfClean.index)] = [m.movieID, m['title'], year, person.personID, person['name']] 
                                elif row.Movie == movie['title'] and found == False:
                                    found = True
                                    for m in works['filmography'][job]:
                                        year = 0000 if 'year' not in m else int(m['year'])
                                        #print(m['title'],'\t', m['year'])
                                        dfClean.loc[len(dfClean.index)] = [m.movieID, m['title'], year, person.personID, person['name']]                                                     
            else:
                continue #print('Person ID:', person.personID, '\tName:    ', person['name'], '\t# Job:', job, "(Doesn't have a filmography)")

In [5]:
dfClean.to_csv("Director_Movies_Clean.csv")

In [288]:
df_movies = pd.read_csv('Director_Movies_Clean.csv', encoding='unicode_escape')

In [8]:
for name in df_director_names.itertuples(): 
    for director in df_movies.itertuples():   
    #print(name.Name)
        if name.Name == director.Name:
            #print(name)
            df_director_names.at[name.Index, 'Person_ID'] = int(director.Person_ID)
                 

#df_director_names

In [144]:
#df_director_names['Person_ID'] = df_director_names['Person_ID'].fillna(0)
df_director_names = df_director_names.astype({"Person_ID": int})
df_director_names

,Unnamed: 0,Name,wiki_link,Person_ID
0,0,Abdisalam Aato,/wiki/Abdisalam_Aato,0
1,3,Omowale Akintunde,/wiki/Omowale_Akintunde,3317997
2,4,Queen Muhammad Ali,/wiki/Queen_Muhammad_Ali,2600662
3,7,M.K. Asante,/wiki/M._K._Asante,1802024
4,8,Philip Atwell,/wiki/Philip_Atwell,0
...,...,...,...,...
306,229,Taylor Ri'chard,/wiki/Taylor_Ri%27chard,6462306
307,1,Fathia Absie,/wiki/Fathia_Absie,7280958
308,6,Madeline Anderson,/wiki/Madeline_Anderson,7507414
309,132,Solomon Sir Jones,/wiki/Solomon_Sir_Jones,8316466


In [301]:
df_director_names.to_csv("director_names.csv", index=False)

In [30]:
ia = Cinemagoer()
person = ia.get_person('0000332')
person['birth date']

'1964-11-29'

#Checking for people without a listing in IMDB

In [154]:
def imdb_search (name):
    imdb_people_search = []
    people = ia.search_person(name)
    for person in people:  
        #print(f" {person.personID} and {person['name']} ")    
        if person['name'] == name:
            imdb_people_search.append([person.personID, person['name']])
    
    if len(imdb_people_search) > 1 or not len(imdb_people_search):
        imdb_people_search = 0

    return imdb_people_search

[['0000332', 'Don Cheadle']]


In [279]:
test_df = df_director_names.loc[df_director_names['Person_ID'] == 0]
test_df

,Unnamed: 0,Name,wiki_link,Person_ID
0,0,Abdisalam Aato,/wiki/Abdisalam_Aato,0
1,15,Mark Beachum,/wiki/Mark_Beachum,0
2,48,Coodie & Chike,/wiki/Coodie_%26_Chike,0
3,80,Ronald Fair,/wiki/Ronald_Fair,0
4,92,Robin Gee,/wiki/Robin_Gee,0
5,94,Leah Gilliam,/wiki/Leah_Gilliam,0
7,121,Hughes brothers,/wiki/Hughes_brothers,0
8,126,Fatima Jamal,/wiki/Fatima_Jamal,0
9,141,L.A. Rebellion,/wiki/L.A._Rebellion,0
12,173,Mary Morten,/wiki/Mary_Morten,0


In [280]:
test_df = test_df.drop("Unnamed: 0", axis=1)
test_df = test_df.reset_index() 
test_df = test_df.drop("index", axis=1)
test_df

,Name,wiki_link,Person_ID
0,Abdisalam Aato,/wiki/Abdisalam_Aato,0
1,Mark Beachum,/wiki/Mark_Beachum,0
2,Coodie & Chike,/wiki/Coodie_%26_Chike,0
3,Ronald Fair,/wiki/Ronald_Fair,0
4,Robin Gee,/wiki/Robin_Gee,0
5,Leah Gilliam,/wiki/Leah_Gilliam,0
6,Hughes brothers,/wiki/Hughes_brothers,0
7,Fatima Jamal,/wiki/Fatima_Jamal,0
8,L.A. Rebellion,/wiki/L.A._Rebellion,0
9,Mary Morten,/wiki/Mary_Morten,0


In [295]:
def update_directors_name(tst_ID, tst_name):
    for name in df_director_names.itertuples(): 
             if name.Name == tst_name:
                #print(name)
                df_director_names.at[name.Index, 'Person_ID'] = int(tst_ID)

In [ ]:
for n in range(0, len(test_df.index)):
    imdb_person = imdb_search(test_df['Name'][n])
    if imdb_person == 0:
        continue
    else:
        person = ia.get_person(imdb_person[0][0])
        if person.has_key('director'):
            for movies in person['filmography']['director']:
                year = 0000 if 'year' not in movies else int(movies['year']) 
                df_movies.loc[len(df_movies.index)] = [movies.movieID, movies['title'], year, imdb_person[0][0], imdb_person[0][1]]
                update_directors_name(imdb_person[0][0], imdb_person[0][1])                            


In [294]:
def wiki_imdb(imdb_id, wiki_name):
    imdb_person = [imdb_id, wiki_name]
    person = ia.get_person(imdb_person[0])
    if person.has_key('director'):
        for movies in person['filmography']['director']:
            year = 0000 if 'year' not in movies else int(movies['year'])
            #print(f"{imdb_person[0]} {person['name']} ")
            df_movies.loc[len(df_movies.index)] = [movies.movieID, movies['title'], year, imdb_person[0], imdb_person[1]]
            update_directors_name(imdb_person[0], imdb_person[1])
            #dfClean.loc[len(dfClean.index)] = [movies.movieID, movies['title'], year, person.personID, person['name']] 

In [299]:
wiki_imdb('10555174', 'Robert Van Lierop')

In [300]:
df_movies.to_csv("Director_Movies_Clean.csv", index=False)

In [ ]:
from bs4 import BeautifulSoup
import requests

imdb_re = 'https://www.imdb.com/name/'
total = 0
for i in range(0, len(test_df.index)):   
    wiki_url = "https://en.wikipedia.org" + test_df['wiki_link'][i]   
    print(wiki_url)  
    table_check = False

    try:
        result = requests.get(wiki_url)
        result.raise_for_status()
        doc = BeautifulSoup(result.text, "html.parser")
    except Exception as e:
        print(e)


    imdb_id = doc.find_all(rel="nofollow")

    for imdb in imdb_id:        
        if re.search(imdb_re, imdb.get('href')):            
            # print(imdb)
            print(imdb.text)
            id = re.search('\d+', imdb.get("href"))
            print(id.group())
            wiki_imdb(id.group(), imdb.text)
            table_check = True            
    if not table_check:        
        total += 1
        print(f"Total: {total} - There wasn't a match for {wiki_url}")


In [302]:
df_movies.describe()

,Year
count,6611.000000
mean,1797.361972
std,601.922601
min,0.000000
25%,1977.000000
50%,2001.000000
75%,2013.000000
max,2027.000000
